# ETF 全时策略：5 只 ETF 等权（每年调仓一次）\n
\n
组合（20% × 5）：\n
- A股进攻：`512050.SH`（中证A500）\n
- A股防守：`159307.SZ`（红利低波100）\n
- 境外股票：`159659.SZ`（纳斯达克100）\n
- 商品：`159937.SZ`（黄金）\n
- 债券：`511130.SH`（30年国债）\n
\n
可选场外基金代码（若你用基金净值口径，需另建数据源）：`023620.OF` / `019125.OF` / `015300.OF` / `000217.OF` / `003613.OF`\n
\n
要点：优先使用复权价（`adj_factor_etf`）；若复权因子覆盖不足，会自动降级为不复权并提示如何补齐数据。\n

In [ ]:
# 环境与依赖（不强依赖 pandas/matplotlib）
import _bootstrap  # noqa: F401 (adds src/ to sys.path)

import datetime as dt
import math

from qs.backtester.runner import (
    load_calendar_bars_for_symbols_from_sqlite,
    run_backtest,
)
from qs.sqlite_utils import connect_sqlite
from qs.strategy.etf_equal_weight_annual import ETFEqualWeightAnnualStrategy


In [ ]:
# 可选：绘图库（若导入失败则仅不画图）
try:
    import matplotlib
    import matplotlib.pyplot as plt

    matplotlib.rcParams["axes.unicode_minus"] = False
except Exception as e:
    matplotlib = None
    plt = None
    print("matplotlib unavailable, skip plotting:", repr(e))


In [ ]:
# 回测参数
DB_PATH = _bootstrap.RAW_DB_PATH  # robust to notebook cwd
SYMBOLS = [
    "512050.SH",  # 中证A500（A股进攻）
    "159307.SZ",  # 红利低波100（A股防守）
    "159659.SZ",  # 纳斯达克100（境外股票）
    "159937.SZ",  # 黄金（商品）
    "511130.SH",  # 30年国债（债券）
]
END_DATE = None  # e.g. "20251231"

INIT_CASH = 1_000_000.0
USE_ADJUSTED = True  # 复权估值 + 复权交易（open/close * adj_factor / base_factor）

if not DB_PATH.exists():
    raise FileNotFoundError(f"Missing DB: {DB_PATH} (run fetch/sync scripts first)")
print("DB:", DB_PATH)

# 自动选择“全时等权”的起始日：取 5 只 ETF 数据的共同起始日（各自 MIN(trade_date) 的最大值）
# 注意：若 USE_ADJUSTED=True，还需要 adj_factor_etf 覆盖到相同区间；否则会将缺失因子当 1.0，导致价格被缩放。
con = connect_sqlite(DB_PATH, read_only=True)
try:
    cover_daily = {}
    cover_adj = {}
    for s in SYMBOLS:
        mn, mx = con.execute(
            "SELECT MIN(trade_date), MAX(trade_date) FROM etf_daily WHERE ts_code=?",
            [s],
        ).fetchone()
        cover_daily[s] = (str(mn) if mn else None, str(mx) if mx else None)
        if USE_ADJUSTED:
            amn, amx = con.execute(
                "SELECT MIN(trade_date), MAX(trade_date) FROM adj_factor_etf WHERE ts_code=?",
                [s],
            ).fetchone()
            cover_adj[s] = (str(amn) if amn else None, str(amx) if amx else None)
finally:
    con.close()

missing_daily = [s for s, (mn, mx) in cover_daily.items() if mn is None]
if missing_daily:
    raise RuntimeError(f"Missing etf_daily data for: {missing_daily}")

START_DATE_DAILY = max(mn for mn, _ in cover_daily.values() if mn is not None)
START_DATE = START_DATE_DAILY

if USE_ADJUSTED:
    missing_adj = [s for s, (mn, mx) in cover_adj.items() if mn is None]
    if missing_adj:
        raise RuntimeError(f"Missing adj_factor_etf data for: {missing_adj}")
    START_DATE_ADJ = max(mn for mn, _ in cover_adj.values() if mn is not None)
    START_DATE = max(START_DATE_DAILY, START_DATE_ADJ)
    if START_DATE > START_DATE_DAILY:
        print("WARN: adj_factor_etf 覆盖不足，会把回测起点推后：")
        print("  START_DATE_DAILY=", START_DATE_DAILY, "START_DATE_ADJ=", START_DATE_ADJ)
        print("  为了拉长回测区间，本次自动切换 USE_ADJUSTED=False。")
        print("  若想继续使用复权价，请先补齐 adj_factor_etf 历史数据（见 README / tushare_sync_daily --rebuild）。")
        USE_ADJUSTED = False
        START_DATE = START_DATE_DAILY

print("Coverage:")
for s in SYMBOLS:
    print(" ", s, cover_daily[s][0], "->", cover_daily[s][1])
if cover_adj:
    print("Adj coverage:")
    for s in SYMBOLS:
        print(" ", s, cover_adj[s][0], "->", cover_adj[s][1])
print("USE_ADJUSTED:", USE_ADJUSTED)
print("START_DATE (overlap):", START_DATE)


In [ ]:
# 运行回测（qs 回测框架）
bars = load_calendar_bars_for_symbols_from_sqlite(
    db_path=DB_PATH,
    table="etf_daily",
    symbols=SYMBOLS,
    start_date=START_DATE,
    end_date=END_DATE,
)
print("calendar bars:", len(bars), "from", bars[0].trade_date, "to", bars[-1].trade_date)

strat = ETFEqualWeightAnnualStrategy(
    db_path_raw=str(DB_PATH),
    symbols=SYMBOLS,
    start_date=START_DATE,
    use_adjusted=USE_ADJUSTED,
    rebalance_year_interval=1,
)

res = run_backtest(
    bars=bars,
    strategy=strat,
    initial_cash=INIT_CASH,
    symbol="",  # 多标的
    enable_trade_log=False,
    mark_error_policy="warn",
)

print(f"Final equity: {res.final_equity:,.2f}  Total return: {(res.final_equity/res.initial_cash-1):.2%}")
print(f"Max DD: {res.max_drawdown:.2%}  ({res.dd_peak} -> {res.dd_trough})")
print("Risk (daily):", dict(res.risk))


In [ ]:
# 策略净值序列（list-based） + 基准对比（纳指/沪深300，开头对齐到 1）
trade_dates = [p.trade_date for p in res.equity_curve]
equities = [float(p.equity) for p in res.equity_curve]
nav = [e / float(INIT_CASH) for e in equities]
dates = [dt.datetime.strptime(d, "%Y%m%d") for d in trade_dates]

print("points:", len(nav), "first:", trade_dates[0], "last:", trade_dates[-1])
print("nav last:", nav[-1])


def load_close_map(table: str, ts_code: str) -> dict[str, float]:
    con = connect_sqlite(DB_PATH, read_only=True)
    try:
        rows = con.execute(
            f"SELECT trade_date, close FROM {table} WHERE ts_code=? AND trade_date>=? ORDER BY trade_date",
            [ts_code, START_DATE],
        ).fetchall()
    finally:
        con.close()
    return {str(d): float(c) for d, c in rows if c is not None}


def align_close(trade_dates: list[str], close_map: dict[str, float]) -> list[float] | None:
    out: list[float] = []
    last = None
    for d in trade_dates:
        if d in close_map:
            last = close_map[d]
        out.append(last if last is not None else float("nan"))
    if not out or all(math.isnan(x) for x in out):
        return None
    first = next((x for x in out if not math.isnan(x)), None)
    if first is None:
        return None
    out = [first if math.isnan(x) else x for x in out]
    return out


hs300_map = load_close_map("index_daily", "000300.SH")
ixic_map = load_close_map("index_global", "IXIC")

hs300_close = align_close(trade_dates, hs300_map)
ixic_close = align_close(trade_dates, ixic_map)

hs300_nav = [c / hs300_close[0] for c in hs300_close] if hs300_close else None
ixic_nav = [c / ixic_close[0] for c in ixic_close] if ixic_close else None

print("HS300 points:", 0 if hs300_nav is None else len(hs300_nav))
print("IXIC points:", 0 if ixic_nav is None else len(ixic_nav))

if plt is not None:
    plt.figure(figsize=(10, 4))
    plt.plot(dates, nav, label="ETF-5 EqualWeight (Annual)")
    if hs300_nav is not None:
        plt.plot(dates, hs300_nav, label="HS300 (000300.SH)")
    if ixic_nav is not None:
        plt.plot(dates, ixic_nav, label="Nasdaq Composite (IXIC)")
    plt.title("Net Asset Value (start=1)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# 调仓记录（每年一次）
hist = strat.get_rebalance_history()
print("rebalance count:", len(hist))
print("rebalance dates (first 10):", [r.rebalance_date for r in hist[:10]])

if hist:
    last = hist[-1]
    print("last rebalance:", last.rebalance_date)
    print("targets sum:", round(sum(last.targets.values()), 6))
    for s in SYMBOLS:
        print(" ", s, "target=", f"{last.targets[s]:.2%}", "open=", f"{last.open_prices[s]:.4f}")
